In [13]:
import re, os
import tiktoken

from bs4 import BeautifulSoup

from langchain_community.document_loaders import RecursiveUrlLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import SKLearnVectorStore
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()


True

In [2]:
def count_tokens(text, model="cl100k_base"):
    """
    Count the number of tokens in the text using tiktoken.

    Args:
        text (str): The text to count tokens for
        model (str): The tokenizer model to use (default: cl100k_base for GPT-4)

    Returns:
        int: Number of tokens in the text
    """
    encoder = tiktoken.get_encoding(model)
    return len(encoder.encode(text))


def bs4_extractor(html: str) -> str:
    """
    Extract text content from HTML using BeautifulSoup.

    Args:
        html (str): Raw HTML content to extract text from

    Returns:
        str: Cleaned text content with excess whitespace removed
    """
    soup = BeautifulSoup(html, "lxml")

    # Target the main article content for LangGraph documentation
    main_content = soup.find("article", class_="md-content__inner")

    # If found, use that, otherwise fall back to the whole document
    content = main_content.get_text() if main_content else soup.text

    # Clean up whitespace
    content = re.sub(r"\n\n+", "\n\n", content).strip()

    return content


def load_langgraph_docs():
    """
    Load LangGraph documentation from the official website.

    This function:
    1. Uses RecursiveUrlLoader to fetch pages from the LangGraph website
    2. Counts the total documents and tokens loaded

    Returns:
        list: A list of Document objects containing the loaded content
        list: A list of tokens per document
    """
    print("Loading LangGraph documentation...")

    # Load the documentation
    urls = [
        "https://langchain-ai.github.io/langgraph/concepts/",
    ]

    docs = []
    for url in urls:

        loader = RecursiveUrlLoader(
            url,
            max_depth=7,
            extractor=bs4_extractor,
        )

        # Load documents using lazy loading (memory efficient)
        docs_lazy = loader.lazy_load()

        # Load documents and track URLs
        for d in docs_lazy:
            docs.append(d)

    print(f"Loaded {len(docs)} documents from LangGraph documentation.")
    print("\nLoaded URLs:")
    for i, doc in enumerate(docs):
        print(f"{i+1}. {doc.metadata.get('source', 'Unknown URL')}")

    # Count total tokens in documents
    total_tokens = 0
    tokens_per_doc = []
    for doc in docs:
        total_tokens += count_tokens(doc.page_content)
        tokens_per_doc.append(count_tokens(doc.page_content))
    print(f"Total tokens in loaded documents: {total_tokens}")

    return docs, tokens_per_doc


def save_llms_full(documents):
    """Save the documents to a file"""

    # Open the output file
    os.makedirs(os.path.join(os.getcwd(), "saved_vector_stores"), exist_ok=True)
    output_filename = os.path.join(os.getcwd(), "saved_vector_stores", "saved_llms_full.txt")

    with open(output_filename, "w") as f:
        # Write each document
        for i, doc in enumerate(documents):
            # Get the source (URL) from metadata
            source = doc.metadata.get("source", "Unknown URL")

            # Write the document with proper formatting
            f.write(f"DOCUMENT {i+1}\n")
            f.write(f"SOURCE: {source}\n")
            f.write("CONTENT:\n")
            f.write(doc.page_content)
            f.write("\n\n" + "=" * 80 + "\n\n")

    print(f"Documents concatenated into {output_filename}")


def split_documents(documents):
    """
    Split documents into smaller chunks for improved retrieval.

    This function:
    1. Uses RecursiveCharacterTextSplitter with tiktoken to create semantically meaningful chunks
    2. Ensures chunks are appropriately sized for embedding and retrieval
    3. Counts the resulting chunks and their total tokens

    Args:
        documents (list): List of Document objects to split

    Returns:
        list: A list of split Document objects
    """
    print("Splitting documents...")

    # Initialize text splitter using tiktoken for accurate token counting
    # chunk_size=8,000 creates relatively large chunks for comprehensive context
    # chunk_overlap=500 ensures continuity between chunks
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=8000, chunk_overlap=500
    )

    # Split documents into chunks
    split_docs = text_splitter.split_documents(documents)

    print(f"Created {len(split_docs)} chunks from documents.")

    # Count total tokens in split documents
    total_tokens = 0
    for doc in split_docs:
        total_tokens += count_tokens(doc.page_content)

    print(f"Total tokens in split documents: {total_tokens}")

    return split_docs

In [8]:
def create_vectorstore(splits):
    """
    Create a vector store from document chunks using SKLearnVectorStore.

    This function:
    1. Initializes an embedding model to convert text into vector representations
    2. Creates a vector store from the document chunks

    Args:
        splits (list): List of split Document objects to embed

    Returns:
        SKLearnVectorStore: A vector store containing the embedded documents
    """
    print("Creating SKLearnVectorStore...")

    # Initialize OpenAI embeddings
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

    # Create vector store from documents using SKLearn
    os.makedirs(os.path.join(os.getcwd(), "saved_vector_stores"), exist_ok=True)
    persist_path = os.getcwd() + "/saved_vector_stores/sklearn_vectorstore.parquet"
    vectorstore = SKLearnVectorStore.from_documents(
        documents=splits,
        embedding=embeddings,
        persist_path=persist_path,
        serializer="parquet",
    )
    print("SKLearnVectorStore created successfully.")

    vectorstore.persist()
    print("SKLearnVectorStore was persisted to", persist_path)

    return vectorstore

In [6]:
# Load the documents
documents, tokens_per_doc = load_langgraph_docs()

# Save the documents to a file
save_llms_full(documents)

# Split the documents
split_docs = split_documents(documents)

Loading LangGraph documentation...
Loaded 41 documents from LangGraph documentation.

Loaded URLs:
1. https://langchain-ai.github.io/langgraph/concepts/
2. https://langchain-ai.github.io/langgraph/concepts/langgraph_self_hosted_data_plane/
3. https://langchain-ai.github.io/langgraph/concepts/langgraph_self_hosted_control_plane/
4. https://langchain-ai.github.io/langgraph/concepts/langgraph_data_plane/
5. https://langchain-ai.github.io/langgraph/concepts/langgraph_data_plane/&
6. https://langchain-ai.github.io/langgraph/concepts/langgraph_cli/
7. https://langchain-ai.github.io/langgraph/concepts/langgraph_control_plane/
8. https://langchain-ai.github.io/langgraph/concepts/langgraph_control_plane/&
9. https://langchain-ai.github.io/langgraph/concepts/application_structure/
10. https://langchain-ai.github.io/langgraph/concepts/langgraph_studio/
11. https://langchain-ai.github.io/langgraph/concepts/sdk/
12. https://langchain-ai.github.io/langgraph/concepts/template_applications/
13. https:

In [9]:
# Create the vector store
vectorstore = create_vectorstore(split_docs)

Creating SKLearnVectorStore...
SKLearnVectorStore created successfully.
SKLearnVectorStore was persisted to c:\workspace\EagSession1\mcp_full_course\llm_codes/saved_vector_stores/sklearn_vectorstore.parquet


In [10]:
# Create retriever to get relevant documents (k=3 means return top 3 matches)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Get relevant documents for the query
query = "What is LangGraph?"
relevant_docs = retriever.invoke(query)
print(f"Retrieved {len(relevant_docs)} relevant documents")

for d in relevant_docs:
    print(d.metadata["source"])
    print(d.page_content[0:500])
    print("\n--------------------------------\n")

Retrieved 3 relevant documents
https://langchain-ai.github.io/langgraph/concepts/high_level/
Why LangGraph?¶
LLM applications¶
LLMs make it possible to embed intelligence into a new class of applications. There are many patterns for building applications that use LLMs. Workflows have scaffolding of predefined code paths around LLM calls. LLMs can direct the control flow through these predefined code paths, which some consider to be an "agentic system". In other cases, it's possible to remove this scaffolding, creating autonomous agents that can plan, take actions via tool calls, and dir

--------------------------------

https://langchain-ai.github.io/langgraph/concepts/faq/
FAQ¶
Common questions and their answers!
Do I need to use LangChain to use LangGraph? What’s the difference?¶
No. LangGraph is an orchestration framework for complex agentic systems and is more low-level and controllable than LangChain agents. LangChain provides a standard interface to interact with models and othe

In [12]:
from langchain_core.tools import tool


@tool
def langgraph_query_tool(query: str):
    """
    Query the LangGraph documentation using a retriever.

    Args:
        query (str): The query to search the documentation with

    Returns:
        str: A str of the retrieved documents
    """
    retriever = SKLearnVectorStore(
        embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
        persist_path=os.getcwd() + "/saved_vector_stores/sklearn_vectorstore.parquet",
        serializer="parquet",
    ).as_retriever(search_kwargs={"k": 3})

    relevant_docs = retriever.invoke(query)
    print(f"Retrieved {len(relevant_docs)} relevant documents")
    formatted_context = "\n\n".join(
        [
            f"==DOCUMENT {i+1}==\n{doc.page_content}"
            for i, doc in enumerate(relevant_docs)
        ]
    )
    return formatted_context

In [15]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm.invoke("What is LangGraph?")

AIMessage(content='As of my last knowledge update in October 2023, LangGraph is not a widely recognized term or concept in mainstream technology, literature, or academia. It could potentially refer to a specific project, tool, or framework that has emerged recently or is niche in nature.\n\nIf LangGraph is a new development, it might be related to natural language processing, graph databases, or a combination of language models and graph theory. For the most accurate and up-to-date information, I recommend checking the latest resources, official documentation, or news articles related to LangGraph. If you have more context or details about what LangGraph pertains to, I would be happy to help clarify further!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 136, 'prompt_tokens': 12, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, '

In [16]:
augmented_llm = llm.bind_tools([langgraph_query_tool])
instructions = """You are a helpful assistant that can answer questions about the LangGraph documentation. 
Use the langgraph_query_tool for any questions about the documentation.
If you don't know the answer, say "I don't know."""

messages = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": "What is LangGraph?"},
]

message = augmented_llm.invoke(messages)
message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  langgraph_query_tool (call_drIZYVyXMaIihX5sX7smvuyE)
 Call ID: call_drIZYVyXMaIihX5sX7smvuyE
  Args:
    query: What is LangGraph?
